In [ ]:
# Step 0: Mount Google Drive (if needed)
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Step 1: Install dependencies and import libraries
!pip install -q transformers
!pip install -q torchinfo
!pip install -q datasets
!pip install -q evaluate
!pip install -q optuna
!pip install -q wandb

import wandb
# Log in to wandb. Replace with your actual WANDB API key
wandb.login(key="your_api_key_here")

# Use DistilBERT instead of Roberta
from transformers import (DistilBertTokenizer, DistilBertForSequenceClassification, Trainer,
                          TrainingArguments)
from datasets import load_dataset, load_from_disk
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 21.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-c

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: peng_zhao (peng_zhao-university-of-california-berkeley) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# Step 2: Define the same tokenization and formatting function used during training
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_and_format(examples):
    outputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)
    outputs["labels"] = [1 if rating > 3 else 0 for rating in examples["rating"]]
    return outputs

# Define evaluation metric computation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
# Step 3a: Load the Grocery_and_Gourmet_Food dataset
food_dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Grocery_and_Gourmet_Food", trust_remote_code=True)

# Use 20% of the dataset
data_fraction = 0.2
total_samples = len(food_dataset["full"])
subset_size = int(total_samples * data_fraction)

# Shuffle and select the specified fraction
subset_dataset = food_dataset["full"].shuffle(seed=42).select(range(subset_size))

# Filter out samples where rating == 3
subset_dataset = subset_dataset.filter(lambda x: x["rating"] != 3)


In [ ]:
# Step 3b: Tokenize the dataset
tokenized_food = food_dataset.map(tokenize_and_format, batched=True)

# Save tokenized checkpoint
tokenized_food.save_to_disk("/content/drive/MyDrive/FP/Checkpoints/tokenized_food_checkpoint")

# Shuffle and format
tokenized_food = tokenized_food.shuffle(seed=42)
tokenized_food.set_format("torch")


In [ ]:
# Step 3': Load saved tokenized data from disk
tokenized_food = load_from_disk("/content/drive/MyDrive/FP/Checkpoints/tokenized_food_checkpoint")
tokenized_food = tokenized_food.shuffle(seed=42)
tokenized_food.set_format("torch")


Loading dataset from disk:   0%|          | 0/46 [00:00<?, ?it/s]

In [ ]:
# Step 4: Load previously fine-tuned DistilBERT model (on Movies_and_TV)
model = DistilBertForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/FP/Checkpoints/final_checkpoint_movies_distilbert", num_labels=2
)


In [ ]:
# Step 5: Initialize Trainer for evaluation (no training)
eval_training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/FP/results_food_eval",
    per_device_eval_batch_size=2048,
    do_predict=True,
    fp16=True,
    report_to=[]  # Disable wandb logging here
)

eval_trainer = Trainer(
    model=model,
    args=eval_training_args,
    compute_metrics=compute_metrics
)


In [ ]:
# Step 6: Evaluate the model on the Grocery_and_Gourmet_Food dataset

# Select 10% of the data for testing
test_fraction = 0.1

# Choose correct dataset split if DatasetDict
if isinstance(tokenized_food, dict):
    dataset_to_sample = tokenized_food["full"]
else:
    dataset_to_sample = tokenized_food

total_samples = len(dataset_to_sample)
test_subset = dataset_to_sample.shuffle(seed=42).select(range(int(test_fraction * total_samples)))

# Perform evaluation
eval_result = eval_trainer.evaluate(eval_dataset=test_subset)
print("Evaluation results on Grocery_and_Gourmet_Food (subset):")
print(eval_result)

# Run predictions
predictions = eval_trainer.predict(test_subset)
print("Predictions summary:")
print(predictions.metrics)

Evaluation results on Grocery_and_Gourmet_Food (subset):
{'eval_loss': 0.23066453635692596, 'eval_model_preparation_time': 0.0017, 'eval_accuracy': 0.908239119685554, 'eval_f1': 0.9036732329117879, 'eval_precision': 0.9081242380082054, 'eval_recall': 0.908239119685554, 'eval_runtime': 1702.9926, 'eval_samples_per_second': 840.786, 'eval_steps_per_second': 0.411}
Predictions summary:
{'test_loss': 0.23066453635692596, 'test_model_preparation_time': 0.0017, 'test_accuracy': 0.908239119685554, 'test_f1': 0.9036732329117879, 'test_precision': 0.9081242380082054, 'test_recall': 0.908239119685554, 'test_runtime': 1485.9573, 'test_samples_per_second': 963.589, 'test_steps_per_second': 0.471}
